In [1]:
import geopandas as gpd
import json
import numpy as np
import plotly.graph_objects as go
import xarray as xr
import urllib.request
import geopandas as gpd

In [ ]:
# Incorporate data
data_location = "/Users/kris/amazonforcast/data/202301/LIS_HIST_2023_Jan.nc"
ds = xr.open_dataset(data_location)

In [ ]:
selected_data = ds['Evap_tavg'].isel(time = 0)
selected_data

In [4]:
#Load and read the geojson file for Italys regions. 
hydrobasins_lev05_url = "https://raw.githubusercontent.com/blackteacatsu/spring_2024_envs_research_amazon_ldas/main/resources/hybas_sa_lev05_areaofstudy.geojson"
with urllib.request.urlopen(hydrobasins_lev05_url) as url:
        jdata = json.loads(url.read().decode())
        

In [6]:
gpd.GeoDataFrame(jdata).set_geometry('coordinates')

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [ ]:
pfaf_id = []
for region in jdata['features']:
    pfaf_id.append(region['properties']['PFAF_ID'])

for region in jdata['features']:
    if region['geometry']['type'] == 'Polygon':
        for p in region["geometry"]["coordinates"][0]:
            print(p[0])

for region in jdata['features']:
    if region['geometry']['type'] == 'MultiPolygon':
        #print("feature is a multipolygon")
        for polyg in region["geometry"]["coordinates"]:
            for p in polyg[0]:
                print(p[0])

In [ ]:
go.Figure(
    [
        go.Scatter(
            **{
                "x": [p[0] for p in region["geometry"]["coordinates"][0]],
                "y": [p[1] for p in region["geometry"]["coordinates"][0]],
                "fill": "toself",
                "name": region['properties']['PFAF_ID'],
            }
        )
        for region in jdata["features"] if region['geometry']['type'] == 'Polygon' 
    ]+

    ## Add multipolygon to the graph
    [
        go.Scatter(
            x=[p[0] for p in polyg[0]],
            y=[p[1] for p in polyg[0]],
            fill="toself",
            name=region['properties']['PFAF_ID'],
        )
        for region in jdata["features"] if region['geometry']['type'] == 'MultiPolygon'
        for polyg in region["geometry"]["coordinates"]
    ]
).update_layout(width=1000, height=900, showlegend=False, margin={"l":0,"r":0,"t":0,"b":0})

In [ ]:
# Initialize the figure
fig = go.Figure()

# Loop through the geojson features
for region in jdata["features"]:
    # Handle Polygons
    if region['geometry']['type'] == 'Polygon':
        fig.add_trace(go.Scatter(
            x=[p[0] for p in region["geometry"]["coordinates"][0]],
            y=[p[1] for p in region["geometry"]["coordinates"][0]],
            fill="toself",
            name=region['properties']['PFAF_ID'],
            line=dict(width=1, color='orange'),
            showlegend=False,
        ))
    
    # Handle MultiPolygons
    elif region['geometry']['type'] == 'MultiPolygon':
        for polyg in region['geometry']['coordinates']:
            # Unpack each polygon and ensure separation with None
            x_coords = [point[0] for point in polyg[0]] + [None]
            y_coords = [point[1] for point in polyg[0]] + [None]
            
            # Add as a new scatter trace
            fig.add_trace(go.Scatter(
                x=x_coords,
                y=y_coords,
                fill="toself",
                name=region['properties']['PFAF_ID'],
                line=dict(width=1, color='orange'),
                showlegend=False,
            ))

# Update layout
fig.update_layout(
    width=1000, 
    height=900, 
    showlegend=False, 
    margin={"l": 0, "r": 0, "t": 0, "b": 0}
)

# Show the figure
fig.show()


In [ ]:
pts = []#list of points defining boundaries of polygons
for feature in jdata['features']:
    if feature['geometry']['type'] == 'Polygon':
        pts.extend(feature['geometry']['coordinates'][0])    
        pts.append([None, None])#mark the end of a polygon   
        
    elif feature['geometry']['type'] == 'MultiPolygon':
        for polyg in feature['geometry']['coordinates']:
            pts.extend(polyg[0])
            pts.append([None, None])#end of polygon
    else: pass           
    #else: raise ValueError("geometry type irrelevant for map")

x, y = zip(*pts) 

fig = go.Figure()
fig.add_scatter(x=x, y=y, mode='lines', line_color='#121212', line_width=1.5, fill='toself')
#fig.add_heatmap(z=selected_data.values, x=selected_data['east_west'], y=selected_data['north_south'])
fig.update_layout(width=1000,  height=900)
fig.show()

In [ ]:
figure = go.Figure()
figure.update_layout(width=1000,  height=900)
figure.add_heatmap(z=selected_data.values, x=selected_data['east_west'], y=selected_data['north_south'], showlegend=False)
for feature in jdata['features']:
    pts = []
    if feature['geometry']['type'] == 'Polygon':
        pts.extend(feature['geometry']['coordinates'][0])    
        pts.append([None, None])#mark the end of a polygon   
        
    elif feature['geometry']['type'] == 'MultiPolygon':
        for polyg in feature['geometry']['coordinates']:
            pts.extend(polyg[0])
            pts.append([None, None])#end of polygon
    elif feature['geometry']['type'] == 'LineString': 
        points.extend(feature['geometry']['coordinates'])
        points.append([None, None])
    else: pass           
    #else: raise ValueError("geometry type irrelevant for map")
    x, y = zip(*pts)
    figure.add_scatter(x=x, y=y, mode='lines', line_color='#121212', line_width=1.5, showlegend=False)

figure.show()

In [ ]:
fig = go.Figure()

# Iterate through features and create scatter traces for each polygon and multipolygon
for feature in jdata['features']:
    # Extract the PFAF_ID for hover labels
    pfaf_id = feature['properties']['PFAF_ID']
    
    # Handle Polygon type
    if feature['geometry']['type'] == 'Polygon':
        # Collect points
        x, y = zip(*feature['geometry']['coordinates'][0])
        # Add scatter trace with hover text
        fig.add_trace(go.Scatter(
            x=x,
            y=y,
            mode='lines',
            line=dict(color='#121212', width=1.5),
            fill='toself',
            text=feature['properties']['PFAF_ID'],  # Add PFAF_ID for each point
            hoverinfo='text'  # Show only the PFAF_ID as hover text
        ))
        # Add None to separate polygons if needed

    # Handle MultiPolygon type
    elif feature['geometry']['type'] == 'MultiPolygon':
        for polyg in feature['geometry']['coordinates']:
            x, y = zip(*polyg[0])
            fig.add_trace(go.Scatter(
                x=x,
                y=y,
                mode='lines',
                line=dict(color='#121212', width=1.5),
                fill='toself',
                text=feature['properties']['PFAF_ID'],  # Add PFAF_ID for each point
                hoverinfo='text'
            ))

# Update layout settings
fig.update_layout(
    width=1000, 
    height=900,
    showlegend=False, 
    margin={"l": 0, "r": 0, "t": 0, "b": 0}
)

# Show figure
fig.show()